In [ ]:
# Connect drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext.data as data
import torchtext
import numpy as np
import pandas as pd


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/NCU/Data Science and Machine Learning/Assignment 3/Datasets/train.csv')
rwtrain = train.drop(train.columns[0], axis=1)

### Text preprocessing

In [ ]:
SENT_LENGTH = 40

In [ ]:
# Define the tokenizer
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

# Define the fields for the dataset
text_field = data.Field(
    sequential=True, 
    use_vocab=True, 
    tokenize=tokenizer, 
    batch_first=True,
    fix_length=SENT_LENGTH)  # set fix_length to the desired length of the sentence

label_field = data.Field(
    sequential=False, use_vocab=False, dtype=torch.float32)

# Create the dataset
fields = [('TEXT', text_field), ('LABEL', label_field)]
examples = [data.Example.fromlist([rwtrain.iloc[i,0], rwtrain.iloc[i,1]], fields) for i in range(len(rwtrain))]
dataset = data.Dataset(examples, fields)



AttributeError: ignored

### GAN architecture (generator/discriminator)

In [ ]:
# Define the generator network
class Generator(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size2)
        self.fc4 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Define the discriminator network
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

### Hyperparameters

In [ ]:
# Set hyperparameters
input_size = 32 # Size of the input data
output_size = 32 # Size of the output data
hidden_size = 128 # Size of the hidden layer
lr = 0.001 # Learning rate
num_epochs = 1000 # Number of training epochs
batch_size = 64 # Number of samples per batch

### Defining loss functions and optimizers

In [ ]:
# Initialize the generator and discriminator
G = Generator(input_size, output_size, hidden_size)
D = Discriminator(output_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.BCELoss()
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)

### Training the GAN

In [ ]:
# Train the GAN (continued)
for epoch in range(num_epochs):
    for i in range(0, len(data), batch_size):
        # Train the discriminator
        D_optimizer.zero_grad()
        real_data = data[i:i+batch_size]
        real_labels = torch.ones(batch_size, 1)
        real_data_tensor = torch.tensor(real_data.values, dtype=torch.float32)
        fake_data = G(torch.randn(batch_size, input_size))
        fake_labels = torch.zeros(batch_size, 1)
        D_real_output = D(real_data_tensor)
        D_fake_output = D(fake_data.detach())
        D_real_loss = criterion(D_real_output, real_labels)
        D_fake_loss = criterion(D_fake_output, fake_labels)
        D_loss = D_real_loss + D_fake_loss
        D_loss.backward()
        D_optimizer.step()
        
        # Train the generator
        G_optimizer.zero_grad()
        fake_data = G(torch.randn(batch_size, input_size))
        fake_labels = torch.ones(batch_size, 1)
        G_output = D(fake_data)
        G_loss = criterion(G_output, fake_labels)
        G_loss.backward()
        G_optimizer.step()

    # Print the loss for every 100th epoch
    if epoch % 100 == 0:
        print('Epoch [{}/{}], D_loss: {:.4f}, G_loss: {:.4f}'.format(epoch+1, num_epochs, D_loss.item(), G_loss.item()))


TypeError: ignored

In [ ]:
# Generate new data with the generator
num_samples = 2000 # Number of samples to generate
generated_data = G(torch.randn(num_samples, input_size)).detach().numpy()

# Combine the original data and the generated data into a single dataframe
original_data = pd.read_csv('original_data.csv') # Replace 'original_data.csv' with the filename of your original data
all_data = pd.DataFrame(np.concatenate((original_data, generated_data), axis=0))
all_data.to_csv('all_data.csv', index=False) # Replace 'all_data.csv' with the filename for the combined data

Epoch 1/100
1/1 [==============================] - 0s 31ms/step


ValueError: ignored